<a href="https://colab.research.google.com/github/Hariharapanda-Deepak/Text-summarization-LLM/blob/main/Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# Path to the zip file
zip_path = '/content/drive/MyDrive/archive.zip'

# Directory to extract the contents
extract_to_path = '/content'

# Unzipping the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"Contents extracted to {extract_to_path}")

Contents extracted to /content


In [ ]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [ ]:
import pandas as pd
dataset = pd.read_csv("/content/cnn_dailymail/train.csv")

In [ ]:
dataset

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."
...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...


In [ ]:
dataset["article"][0]

"Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for sp

In [ ]:
!pip install keybert

In [ ]:
from keybert import KeyBERT
import nltk
from nltk.corpus import stopwords
import string

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Initialize KeyBERT model
kw_model = KeyBERT()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
# Initialize text summarization model
summarizer = pipeline("summarization", model="Falconsai/text_summarization")

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Preprocessing: Prepare data for fine-tuning
train_data = []
for index, row in dataset.iterrows():
    train_data.append({
        "input_text": row['article'],  # Article
        "target_text": row['highlights']  # Highlights as the summary
    })

In [ ]:
train_data[1]

{'input_text': "A drunk teenage boy had to be rescued by security after jumping into a lions' enclosure at a zoo in western India. Rahul Kumar, 17, clambered over the enclosure fence at the\xa0Kamla Nehru Zoological Park in Ahmedabad, and began running towards the animals, shouting he would 'kill them'. Mr Kumar explained afterwards that he was drunk and 'thought I'd stand a good chance' against the predators. Next level drunk: Intoxicated Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in Ahmedabad and began running towards the animals shouting 'Today I kill a lion!' Mr Kumar had been sitting near the enclosure when he suddenly made a dash for the lions, surprising zoo security. The intoxicated teenager ran towards the lions, shouting: 'Today I kill a lion or a lion kills me!' A zoo spokesman said: 'Guards had earlier spotted him close to the enclosure but had no idea he was planing to enter it. 'Fortunately, there are eight moats to cross before getting to where the lions

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.6 MB/s eta 0:00:00


In [ ]:
df=dataset

In [ ]:
df

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."
...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset, concatenate_datasets

# Load the tokenizer and model
model_name = "Falconsai/text_summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define the tokenization function
def tokenize_function(examples):
    # Tokenize input and target text
    model_inputs = tokenizer(
        examples['article'], max_length=512, truncation=True, padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['highlights'], max_length=150, truncation=True, padding="max_length"
        )
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Load and preprocess the dataset
dataset_chunks = []
chunk_size = 500  # Adjust based on available memory
for i in range(0, len(dataset), chunk_size):
    chunk = dataset.iloc[i:i + chunk_size]
    chunk_ds = Dataset.from_pandas(chunk[['article', 'highlights']])
    chunk_ds = chunk_ds.map(tokenize_function, batched=True, batch_size=32)
    dataset_chunks.append(chunk_ds)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/490 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
# Combine tokenized chunks
train_dataset = concatenate_datasets(dataset_chunks)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./summarization_model",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    save_steps=100,
    save_total_limit=2,
    weight_decay=0.01,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100,
)

# Trainer initialization
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,2.637700
200,1.124400
300,1.049600
400,1.060300
500,1.032700
600,1.047700
700,1.057500
800,1.013700
900,1.031200
1000,1.016100


TrainOutput(global_step=17945, training_loss=1.0128962459654554, metrics={'train_runtime': 10557.5586, 'train_samples_per_second': 27.195, 'train_steps_per_second': 1.7, 'total_flos': 3.885839064603034e+16, 'train_loss': 1.0128962459654554, 'epoch': 1.0})

In [ ]:
# Save the final model and tokenizer
model.save_pretrained("/content/drive/MyDrive/summarization_model")
tokenizer.save_pretrained("/content/drive/MyDrive/summarization_model")

('/content/drive/MyDrive/summarization_model/tokenizer_config.json',
 '/content/drive/MyDrive/summarization_model/special_tokens_map.json',
 '/content/drive/MyDrive/summarization_model/spiece.model',
 '/content/drive/MyDrive/summarization_model/added_tokens.json',
 '/content/drive/MyDrive/summarization_model/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Path to the model in Google Drive
model_path = "/content/drive/MyDrive/summarization_model"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# Path to the zip file
zip_path = '/content/drive/MyDrive/Untitled folder/archive (1).zip'

# Directory to extract the contents
extract_to_path = '/content'

# Unzipping the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"Contents extracted to {extract_to_path}")

Contents extracted to /content


In [ ]:
import zipfile
import os

# Path to the zip file
zip_path = '/content/drive/MyDrive/Untitled folder/text summary (1)-20250220T074423Z-001.zip'

# Directory to extract the contents
extract_to_path = '/content'

# Unzipping the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"Contents extracted to {extract_to_path}")

Contents extracted to /content


In [ ]:
import pandas as pd
dataset = pd.read_csv("/content/cnn_dailymail/test.csv")

In [ ]:
dataset

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."
...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...


In [ ]:
dataset["article"][0]

"Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for sp

In [ ]:
!pip install keybert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 836.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
from keybert import KeyBERT
import nltk
from nltk.corpus import stopwords
import string

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Initialize KeyBERT model
kw_model = KeyBERT()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import os

model_path = "/content/text summary (1)"

# List all files in the model directory
files = os.listdir(model_path)

# Print the list of files
print("Files in model directory:", files)

Files in model directory: ['model.safetensors', 'generation_config (2).json', 'rng_state (2).pth', 'tokenizer.json', 'optimizer (1).pt', 'spiece (2).model', 'trainer_state (2).json', 'special_tokens_map (2).json', 'config.json', 'scheduler (2).pt', 'tokenizer_config (2).json', 'training_args (2).bin']


In [ ]:
import os
import shutil

model_path = "/content/text summary (1)"

# Define expected filenames
expected_files = {
    "config (2).json": "config.json",
    "tokenizer (2).json": "tokenizer.json",
    "model (2).safetensors": "model.safetensors",
}

# Rename the files
for old_name, new_name in expected_files.items():
    old_path = os.path.join(model_path, old_name)
    new_path = os.path.join(model_path, new_name)

    if os.path.exists(old_path):
        shutil.move(old_path, new_path)
        print(f"Renamed: {old_name} → {new_name}")

# Verify the files after renaming
print("Updated Files:", os.listdir(model_path))

Updated Files: ['model.safetensors', 'generation_config (2).json', 'rng_state (2).pth', 'tokenizer.json', 'optimizer (1).pt', 'spiece (2).model', 'trainer_state (2).json', 'special_tokens_map (2).json', 'config.json', 'scheduler (2).pt', 'tokenizer_config (2).json', 'training_args (2).bin']


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Define the path to your saved model in Google Drive
model_path = "/content/text summary (1)"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Initialize the summarization pipeline
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [ ]:
# Function to clean and preprocess text
def clean_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ' '.join(word for word in text.split() if word.lower() not in stop_words)
    return text

# Function for keyword extraction
def extract_keywords(text):
    keywords = kw_model.extract_keywords(text, top_n=5)
    return ', '.join([kw[0] for kw in keywords])

# Function for text summarization
def summarize_text(text):
    summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
    return summary[0]['summary_text']

In [ ]:
df=dataset

In [ ]:
from nltk.corpus import stopwords
# Apply keyword extraction and summarization
df['cleaned_article'] = df['article'].apply(clean_text)

In [ ]:
df

,id,article,highlights,cleaned_article
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,Ever noticed plane seats appear getting smalle...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,drunk teenage boy rescued security jumping lio...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,Dougie Freedman verge agreeing new twoyear dea...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,Liverpool target Neto also wanted PSG clubs Sp...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...",Bruce Jenner break silence twohour interview D...
...,...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...,young Earth may collided body similar planet M...
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...,man facing trial helping former lover strangle...
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...,dozen metal implements arranged neat rows tabl...
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...,Brook Lopez dominated twin brother Robin 32 po...


In [ ]:
df['keywords'] = df['cleaned_article'].apply(extract_keywords)

In [ ]:
df

,id,article,highlights,cleaned_article,keywords
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,Ever noticed plane seats appear getting smalle...,"airlines, passengers, seats, aeroplanes, planes"
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,drunk teenage boy rescued security jumping lio...,"zoo, predators, zoological, lion, lions"
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,Dougie Freedman verge agreeing new twoyear dea...,"pearce, freedman, forest, contract, dougie"
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,Liverpool target Neto also wanted PSG clubs Sp...,"liverpool, neto, atletico, juventus, goalkeeper"
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...",Bruce Jenner break silence twohour interview D...,"jenner, jenners, kardashian, kardashians, kyle"
...,...,...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...,young Earth may collided body similar planet M...,"meteorites, planets, mercury, asteroids, mercu..."
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...,man facing trial helping former lover strangle...,"duffys, duffy, murdered, victims, accused"
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...,dozen metal implements arranged neat rows tabl...,"facials, forks, facial, forehead, metal"
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...,Brook Lopez dominated twin brother Robin 32 po...,"blazers, brook, lillard, lopez, lamarcus"


In [ ]:
df['summary'] = df['article'].apply(summarize_text)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
df

,id,article,highlights,cleaned_article,keywords,summary
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,Ever noticed plane seats appear getting smalle...,"airlines, passengers, seats, aeroplanes, planes",A U.S consumer advisory group set up by the De...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,drunk teenage boy rescued security jumping lio...,"zoo, predators, zoological, lion, lions","Rahul Kumar, 17, climbed into enclosure fence ..."
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,Dougie Freedman verge agreeing new twoyear dea...,"pearce, freedman, forest, contract, dougie",Dougie Freedman is set to sign a new two-year ...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,Liverpool target Neto also wanted PSG clubs Sp...,"liverpool, neto, atletico, juventus, goalkeeper",Neto is wanted by a number of top European clu...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...",Bruce Jenner break silence twohour interview D...,"jenner, jenners, kardashian, kardashians, kyle","The former Olympian and reality TV star, 65, w..."
...,...,...,...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...,young Earth may collided body similar planet M...,"meteorites, planets, mercury, asteroids, mercu...",Oxford scientists say a Mercury-like body stru...
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...,man facing trial helping former lover strangle...,"duffys, duffy, murdered, victims, accused","Micheal John Duffy, 34, pleaded not guilty to ..."
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...,dozen metal implements arranged neat rows tabl...,"facials, forks, facial, forehead, metal",The tuning forks are made of metal that can ti...
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...,Brook Lopez dominated twin brother Robin 32 po...,"blazers, brook, lillard, lopez, lamarcus",Brooklyn Nets beat Portland Trail Blazers on M...


In [ ]:
df.to_csv("dataset.csv", index=False)  # Saves without the index column

In [ ]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=164e1969e0d9c214e8f70c9f00cd1207dab13313780575eeef5a3ffc7b719507
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
from rouge_score import rouge_scorer

# Create a ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate ROUGE scores for each row
rouge_scores = []
for _, row in df.iterrows():
    highlight = row['highlights']
    summary = row['summary']

    scores = scorer.score(highlight, summary)
    rouge_scores.append(scores)

# Add the ROUGE scores to the dataframe
df['rouge_scores'] = rouge_scores

# Print out the dataframe with ROUGE scores
print(df[['highlights', 'summary', 'rouge_scores']])

                                              highlights  \
0      Experts question if  packed out planes are put...   
1      Drunk teenage boy climbed into lion enclosure ...   
2      Nottingham Forest are close to extending Dougi...   
3      Fiorentina goalkeeper Neto has been linked wit...   
4      Tell-all interview with the reality TV star, 6...   
...                                                  ...   
11485  Oxford scientists say a Mercury-like body stru...   
11486  Man accused of helping former lover kill woman...   
11487  Marianne Power tried the tuning fork facial at...   
11488  Brooklyn Nets beat the Portland Trail Blazers ...   
11489  Chinese hospital marked for demolition because...   

                                                 summary  \
0      A U.S consumer advisory group set up by the De...   
1      Rahul Kumar, 17, climbed into enclosure fence ...   
2      Dougie Freedman is set to sign a new two-year ...   
3      Neto is wanted by a number of to

In [ ]:
df

,id,article,highlights,cleaned_article,keywords,summary,rouge_scores
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,Ever noticed plane seats appear getting smalle...,"airlines, passengers, seats, aeroplanes, planes",A U.S consumer advisory group set up by the De...,"{'rouge1': (0.2619047619047619, 0.323529411764..."
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,drunk teenage boy rescued security jumping lio...,"zoo, predators, zoological, lion, lions","Rahul Kumar, 17, climbed into enclosure fence ...","{'rouge1': (0.72, 0.5, 0.5901639344262295), 'r..."
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,Dougie Freedman verge agreeing new twoyear dea...,"pearce, freedman, forest, contract, dougie",Dougie Freedman is set to sign a new two-year ...,"{'rouge1': (0.4, 0.36363636363636365, 0.380952..."
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,Liverpool target Neto also wanted PSG clubs Sp...,"liverpool, neto, atletico, juventus, goalkeeper",Neto is wanted by a number of top European clu...,"{'rouge1': (0.3684210526315789, 0.35, 0.358974..."
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...",Bruce Jenner break silence twohour interview D...,"jenner, jenners, kardashian, kardashians, kyle","The former Olympian and reality TV star, 65, w...","{'rouge1': (0.6470588235294118, 0.366666666666..."
...,...,...,...,...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...,young Earth may collided body similar planet M...,"meteorites, planets, mercury, asteroids, mercu...",Oxford scientists say a Mercury-like body stru...,"{'rouge1': (0.868421052631579, 0.7333333333333..."
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...,man facing trial helping former lover strangle...,"duffys, duffy, murdered, victims, accused","Micheal John Duffy, 34, pleaded not guilty to ...","{'rouge1': (0.75, 0.25301204819277107, 0.37837..."
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...,dozen metal implements arranged neat rows tabl...,"facials, forks, facial, forehead, metal",The tuning forks are made of metal that can ti...,"{'rouge1': (0.17647058823529413, 0.18181818181..."
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...,Brook Lopez dominated twin brother Robin 32 po...,"blazers, brook, lillard, lopez, lamarcus",Brooklyn Nets beat Portland Trail Blazers on M...,"{'rouge1': (0.41025641025641024, 0.44444444444..."


In [ ]:
import pandas as pd

# Extract F1 scores from tuples in each ROUGE column
df[['rouge1_f1', 'rouge2_f1', 'rougeL_f1']] = df[['rouge1', 'rouge2', 'rougeL']].applymap(lambda x: x[2] if isinstance(x, tuple) else None)

# Calculate mean ROUGE scores
mean_rouge1 = df['rouge1_f1'].mean()
mean_rouge2 = df['rouge2_f1'].mean()
mean_rougeL = df['rougeL_f1'].mean()

print(f"Mean ROUGE-1 F1: {mean_rouge1}")
print(f"Mean ROUGE-2 F1: {mean_rouge2}")
print(f"Mean ROUGE-L F1: {mean_rougeL}")


Mean ROUGE-1 F1: 0.6416666666666667
Mean ROUGE-2 F1: 0.43333333333333335
Mean ROUGE-L F1: 0.5916666666666667


<ipython-input-1-af87a2952bbc>:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[['rouge1_f1', 'rouge2_f1', 'rougeL_f1']] = df[['rouge1', 'rouge2', 'rougeL']].applymap(lambda x: x[2] if isinstance(x, tuple) else None)


In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
pip install keybert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e7767c9bbb204af9f764ba20a44317ea90243b92718a748b3f9e65be050c3809
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
import gradio as gr
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
from nltk.corpus import stopwords
import string
from keybert import KeyBERT
import torch
from rouge_score import rouge_scorer  # Import ROUGE scorer

# Download NLTK stopwords if not already downloaded
try:
    stop_words = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

# Use a lightweight model for summarization (e.g., BART or T5)
summarization_model = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(summarization_model)

# Load the model without specifying device_map initially
model = AutoModelForSeq2SeqLM.from_pretrained(summarization_model, torch_dtype=torch.float16)

# Move the model to the desired device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=device if device.type == "cuda" else -1)

# Initialize KeyBERT model
kw_model = KeyBERT()

# Function to clean and preprocess text
def clean_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = ' '.join(word for word in text.split() if word.lower() not in stop_words)  # Remove stopwords
    return text

# Function for keyword extraction
def extract_keywords(text):
    keywords = kw_model.extract_keywords(text, top_n=5)
    return ', '.join([kw[0] for kw in keywords])

# Function for text summarization using a lightweight model
def summarize_text(text):
    summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
    return summary[0]['summary_text']

# Function to compute ROUGE score
def compute_rouge(reference, generated):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return {
        "ROUGE-1": round(scores["rouge1"].fmeasure, 4),
        "ROUGE-2": round(scores["rouge2"].fmeasure, 4),
        "ROUGE-L": round(scores["rougeL"].fmeasure, 4)
    }

# Function to process text and compute ROUGE scores
def process_text(text):
    cleaned_text = clean_text(text)
    keywords = extract_keywords(cleaned_text)
    summary = summarize_text(text)

    # Compute ROUGE scores
    rouge_summary = compute_rouge(text, summary)

    return cleaned_text, keywords, summary, str(rouge_summary)

# Gradio interface
iface = gr.Interface(
    fn=process_text,
    inputs=gr.Textbox(lines=10, placeholder="Enter your text here..."),
    outputs=[
        gr.Textbox(label="Cleaned Text"),
        gr.Textbox(label="Keywords"),
        gr.Textbox(label="Summary"),
        gr.Textbox(label="ROUGE Score (Summary vs. Original)")
    ],
    title="Text Summarizer & Keyword Extractor with ROUGE Score",
    description="Enter text to generate a summary and extract keywords using KeyBERT. ROUGE scores measure similarity with the original text.",
)

iface.launch(debug=True)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f88eeb1d1084d0003f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
